### To-do List
1) ~~Need to extract agent-id from pickle files to a variable~~\
2) ~~Need to extract track-id from pickle files to a variable~~\
3) ~~Need to extract car mask from pickle files to a variable~~\
4) Need to use extracted agent-id and match with the correct track-id to monitor that track-id's predictions\
5) ~~Need to use car mask to get only the track-id's that have objects~~

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
import math
import csv
from glob import glob

"""Change to the data folder"""
train_path = "./new_train/new_train"
val_path = "./new_val_in/new_val_in"
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)
            
        return data


# intialize a dataset
train_dataset = ArgoverseDataset(data_path=train_path)
val_dataset  = ArgoverseDataset(data_path=val_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 32

### train_collate definition for train_loader

In [4]:


def train_collate(batch): 
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    #print(batch[0]['car_mask'])
    agent_ids = [scene['agent_id'] for scene in batch]
    #agent_id = batch[0]['agent_id']
    car_masks = [scene['car_mask'] for scene in batch]
    #car_mask = batch[0]['car_mask']
    track_ids = [scene['track_id'] for scene in batch]
    
    #actual_objects = track_id[car_mask.reshape([-1]).astype(int)] #track id actually used
    actual_objects = []
    actual_obj = []
    agent_indices = []
    
    for c_mask,t_id in zip(car_masks,track_ids): #use only tracks that have vehicles on it, if no vehicle present it's labelled as -1
        for mask,track in zip(c_mask,t_id):
            if mask[0] == 1:
                actual_obj.append(track[0][0])
            else:
                actual_obj.append(-1)
        actual_objects.append(actual_obj)
        actual_obj = []
    
    for (i,track),agent_id in zip(enumerate(actual_objects),agent_ids): #will look through the tracks with vehicles and see which one is the agent's track
        for j,tr in enumerate(track):
    
            if tr == agent_id:
                agent_index = j
                break
        agent_indices.append(agent_index)
        #print("Confirmation of correct agent_index",agent_id," : ",track[agent_index])
    
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scenes_indices = [scene['scene_idx'] for scene in batch]
    return [inp, out,scenes_indices,agent_indices]

### val_collate definition for val_loader

In [5]:
def val_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = torch.LongTensor(inp)
    
    agent_ids = [scene['agent_id'] for scene in batch]
    car_masks = [scene['car_mask'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    
    actual_objects = [] #track id actually used
    actual_obj = []
    agent_indices = []
    
    for c_mask,t_id in zip(car_masks,track_ids): #use only tracks that have vehicles on it, if no vehicle present it's labelled as -1
        for mask,track in zip(c_mask,t_id):
            if mask[0] == 1:
                actual_obj.append(track[0][0])
            else:
                actual_obj.append(-1)
        actual_objects.append(actual_obj)
        actual_obj = []
    
    for (i,track),agent_id in zip(enumerate(actual_objects),agent_ids): #will look through the tracks with vehicles and see which one is the agent's track
        for j,tr in enumerate(track):
            if tr == agent_id:
                agent_index = j
                break
        agent_indices.append(agent_index)
    
    inp = torch.LongTensor(inp)

    scenes_indices = [scene['scene_idx'] for scene in batch]
    return [inp, scenes_indices,agent_indices]

### Set train_loader and val_loader

In [6]:
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=val_collate, num_workers=0)

### CNN Model

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class CNN(nn.Module):
    """This class defines your deep learning model that extends a Module class
      The constructor of your class defines the layers of the model. 
      The forward() function defines how to forward propagate 
      input through the defined layers of the model.
      Many layers are available, such as Linear for fully connected layers, 
      Conv2d for convolutional layers, and MaxPool2d for pooling layers.

    """
    #============================================
    # TODO: Implement CNN model.
    #=============================================
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4,100,3) #it's 4*60 because there's 60 vehicles and we're just looking at it's present features [pos_x,pos_y,vel_x,vel_y]
        self.pool = nn.MaxPool2d(2, 2)
        self.batchNorm1 = nn.BatchNorm2d(100)
        self.conv2 = nn.Conv2d(100,60,1)
        self.fc1 = nn.Linear(60 * 19 * 4, 120)
        self.batchNorm2 = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 60 * 30 * 4)
        self.softMax = nn.LogSoftmax()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    '''
        1) Understand dataset correctly (input shape and output shape etc.)
        2) 
    '''
    def forward(self, x):
        x = x.to(self.device)
        #x = self.pool(F.relu(self.conv1(x)))#F.relu(self.conv1(x))
        #x = self.batchNorm1(x)
        #print("xshape in forward after batchNorm1: ",x.shape)
        #x = self.pool(F.relu(self.conv2(x)))
        #print("xshape in forward after conv2: ",x.shape)
        
        #x = x.view(-1, 60 * 19 * 4)
        x = F.relu(self.fc1(x))
        #print("xshape in forward after one fc1: ",x.shape)
        #x = self.batchNorm2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #x = self.softMax(x) #Need to add this because training and testing use nll_loss
        return x

### define Train function

In [8]:
from tqdm import tqdm_notebook as tqdm
def train(model, device, train_loader, optimizer, epoch,log_interval=10000):
    model.train()
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    inp_arr_posx_train = []
    inp_arr_posy_train = []
    out_arr_posx_train = []
    out_arr_posy_train= []
    
    for batch_idx, (data,target,scenes_indices,agent_indices) in enumerate(iterator):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        
        if epoch == 1:
            for data_scene,target_scene in zip(data,target):
                for data_vehicle,target_vehicle in zip(data_scene,target_scene):
                    inp_arr_posx_train.append(data_vehicle[18][0])
                    inp_arr_posy_train.append(data_vehicle[18][1])
                    out_arr_posx_train.append(target_vehicle[18][0])
                    out_arr_posy_train.append(target_vehicle[18][1])
            
        
        data = data.type(torch.FloatTensor)
        data = torch.reshape(data,(train_loader.batch_size,60 * 19 * 4))
        #data = torch.reshape(data,(1,4,60,19)) #used with convolutional layers
        target = target.type(torch.FloatTensor)
        target = torch.reshape(target,(train_loader.batch_size,60 * 30 * 4)) 
        #print("data in train tranposed:",data.shape)
        output = model(data)
        output = output.to(device)
        outputPickleFormat = torch.reshape(output,(train_loader.batch_size,60,30,4))
        #print(outputPickleFormat[0][agent_index])
        
        target = target.to(device)
        #output = torch.reshape(output,(1,60 * 30 * 4))
        targetPickleFormat = torch.reshape(target,(train_loader.batch_size,60,30,4))
        #print(targetPickleFormat[0][agent_index])

        #calculate loss with respect to the agent's movements
        runningLoss = 0
        for batch_i,agent_index in enumerate(agent_indices):
            loss = torch.sqrt(torch.mean((outputPickleFormat[batch_i][agent_index]-targetPickleFormat[batch_i][agent_index])**2))
            runningLoss += loss
            
        loss = runningLoss / len(agent_indices)
        #print(loss)
        loss.backward()
        optimizer.step()
        counter += 1
        #print(loss.item())
        iterator.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))
        #print(loss.item())
    #histogram for x and y input positions of train's agents
    plt.style.use('fivethirtyeight')
    plt.hist(inp_arr_posx_train,edgecolor='black')
    plt.title("Input x-axis distribution for all agents in training")
    plt.xlabel("x-position")
    plt.ylabel("frequency")
    plt.show()

    plt.style.use('fivethirtyeight')
    plt.hist(inp_arr_posy_train,edgecolor='black')
    plt.title("Input y-axis distribution for all agents in training")
    plt.xlabel("y-position")
    plt.ylabel("frequency")
    plt.show()

    #histogram for x and y output positions of train's agents
    plt.style.use('fivethirtyeight')
    plt.hist(out_arr_posx_train,edgecolor='black')
    plt.title("Output x-axis distribution for all agents in training")
    plt.xlabel("x-position")
    plt.ylabel("frequency")
    plt.show()

    plt.style.use('fivethirtyeight')
    plt.hist(out_arr_posy_train,edgecolor='black')
    plt.title("Output y-axis distribution for all agents in training")
    plt.xlabel("y-position")
    plt.ylabel("frequency")
    plt.show()
'''data = train_loader.to(device)
    target = target_loader.to(device)
    optimizer.zero_grad()
    
    #data = data.type(torch.FloatTensor)
    output = model(data)
    print(output.shape)
    print(output)
    loss = torch.mean((output-target)**2)
    loss.backward()
    optimizer.step()
    print(output)'''

'data = train_loader.to(device)\n    target = target_loader.to(device)\n    optimizer.zero_grad()\n    \n    #data = data.type(torch.FloatTensor)\n    output = model(data)\n    print(output.shape)\n    print(output)\n    loss = torch.mean((output-target)**2)\n    loss.backward()\n    optimizer.step()\n    print(output)'

### define Test function

In [9]:
def test(model, device, test_loader,epoch):#target_loader,epoch):
    model.eval()
    test_loss = 0
    predictedPos = []
    predPosArr = []
    correct = 0
    
    with torch.no_grad():
        for data,scenes_indices,agent_indices in test_loader:
            
            data = data.to(device)
            data = data.type(torch.FloatTensor)
            #data = torch.reshape(data,(1,4,60,19))
            data = torch.reshape(data,(test_loader.batch_size,60 * 19 * 4))
            output = model(data)
            #print(output)
            output = torch.reshape(output,(test_loader.batch_size,60 * 30 * 4))
            outputPickleFormat = torch.reshape(output,(test_loader.batch_size,60,30,4))
            #print(outputPickleFormat)
            
            for batch,scene_index in zip(outputPickleFormat,scenes_indices):
                for vehicle in batch:
                    predicted_x = vehicle[29][0] #should give us the predicted 30th time step's pos x
                    predicted_y = vehicle[29][1] #should give us the predicted 30th time step's pos y
                    predictedPos.append(math.sqrt((predicted_x**2) + (predicted_y**2)))
                predictedPos.insert(0,scene_index)
                scenes_indices.remove(scene_index)
                predPosArr.append(predictedPos)
                predictedPos = []
        
    return predPosArr
    

### Write CSV header

In [10]:
import matplotlib.pyplot as plt
import random

with open('submission.csv','w',newline='') as f:
    thewriter = csv.writer(f)
    headerRow = []
    
    headerRow.append("ID")
    for i in range(1,61):
        headerRow.append("v" + str(i))
        
    thewriter.writerow(headerRow)

### show_sample_batch function definition

In [11]:
agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out,scenes_indices = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

### Start training

In [12]:
learning_rate = 0.001
momentum = 0.5
if torch.cuda.is_available():
    print("cuda chosen")
    dev = "cuda"
else:
    print("cpu chosen")
    dev = "cpu"
        
device = dev
model = CNN().to(device) #using cpu here
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
num_epoch = 10

print(len(train_loader))
for epoch in range(1, num_epoch + 1):
    #for i_batch, train_batch in enumerate(train_loader): #Begin training
        #inp_train, out_train, scenes_indices_train,agent_indices_train = train_batch
    

    predictedList = []

    train(model, device, train_loader, optimizer, epoch) #replace train_loader with our input training data
    predictedList = test(model, device, val_loader,epoch)   

    if epoch == num_epoch: #if done with the last epoch then we write to file
        with open('submission.csv','a',newline='') as f: #append to csv
            thewriter = csv.writer(f)
            for p_list in predictedList:
                thewriter.writerow(p_list)
    '''plt.style.use('fivethirtyeight')
    plt.plot(loss_arr,'r')
    plt.title("Loss graph")
    plt.xlabel("indices")
    plt.ylabel("loss")
    plt.show()'''
    
    print("Done with one epoch")


    
    
#for j_batch,val_batch in enumerate(val_loader): #Begin testing
    #inp_val, scenes_indices_val,agent_indices_val = val_batch

    #print(scenes_indices)
    #print(sample_batch)

                          #momentum=momentum)#,weight_decay=1)
    #rowList = []
    #batchList = []

    #for inp_arr_val,scene_idx_val,agent_idx_val in zip(inp_val,scenes_indices_val,agent_indices_val):
       # rowList = test(model, device, inp_arr_val,epoch,agent_idx_val)#,out_arr_test,epoch) #replace test_loader with our input test data
        #rowList.insert(0,scene_idx_val)
        #if(epoch == num_epoch): #if at the last epoch then we'll add onto the list that we'll write to csv
            #batchList.append(rowList)

    #if(epoch == num_epoch):
        #with open('submission.csv','a',newline='') as f: #append to csv
            #thewriter = csv.writer(f)
            #for predictedList in batchList:
                #thewriter.writerow(predictedList)


    #Show sample batch on graph
    #show_sample_batch(train_batch, agent_id)
    #write to csv

    
    
    
    
print("Done with training")


cuda chosen
6436


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.92 GiB total capacity; 9.93 GiB already allocated; 17.50 MiB free; 358.04 MiB cached)